In [1]:
# POC for ny taxi

In [190]:
import pandas as pd
import glob
from pandasql import sqldf

In [108]:
# helpers
pysqldf = lambda q: sqldf(q, globals())

In [191]:
data_file_location = '../resources/data/enriched_trip_data.parquet'

## Loading enriched preprocessed data

In [201]:
# trip data
all_trip_data_df=pd.read_parquet(data_file_location)
all_trip_data_df = all_trip_data_df.dropna()


In [210]:
all_trip_data_df = all_trip_data_df.dropna()

all_trip_data_df.count()

tpep_pickup_datetime     6922978
tpep_dropoff_datetime    6922978
passenger_count          6922978
pickup_borough           6922978
pickup_zone              6922978
drop_off_borough         6922978
drop_off_zone            6922978
month                    6922978
dtype: int64

In [208]:
grouped[(grouped["pickup_zone"] == "Chinatown") ].sort_values(by=['month'])

,count,pickup_zone,drop_off_zone,month
4909,45,Chinatown,None,2019-02
5042,26,Chinatown,Newark Airport,2019-02
5043,2,Chinatown,North Corona,2019-02
5044,11,Chinatown,Norwood,2019-02
5045,1,Chinatown,Oakland Gardens,2019-02
...,...,...,...,...
32696,5,Chinatown,Crown Heights North,2019-03
32697,1,Chinatown,Financial District North,2019-03
32698,1,Chinatown,Greenwich Village South,2019-03
32699,2,Chinatown,Kips Bay,2019-03


## Prepare Data according to Task 2.i

In [203]:
grouped = pysqldf("SELECT sum(passenger_count) as count, pickup_zone, drop_off_zone, month FROM all_trip_data_df GROUP BY month, pickup_zone, drop_off_zone")
grouped


,count,pickup_zone,drop_off_zone,month
0,1,Clinton East,TriBeCa/Civic Center,2008-12
1,6,Clinton West,Murray Hill,2008-12
2,2,Financial District South,Meatpacking/West Village West,2008-12
3,1,Gramercy,World Trade Center,2008-12
4,1,JFK Airport,Hamilton Heights,2008-12
...,...,...,...,...
34857,1,JFK Airport,Saint Albans,2019-09
34858,1,Greenwich Village North,Upper West Side South,2038-02
34859,1,West Village,Greenwich Village South,2038-02
34860,1,Yorkville East,Lenox Hill East,2038-02


In [207]:
grouped[(grouped["pickup_zone"] == "Chinatown") & (grouped["month"] == "2019-09") ].sort_values(by=['count'])

,count,pickup_zone,drop_off_zone,month


In [211]:
# rank by k=50 per pickup_zone and month
ranked = grouped.copy(deep=True)
ranked["rank"]=ranked.groupby(["pickup_zone","month"])["count"].rank(method="max", ascending=False)
ranked = ranked[ranked["rank"] <= 50.0]
#ranked[(ranked["pickup_zone"] == "Chinatown") & (ranked["month"] == "2019-10")].sort_values(by=['rank'])
ranked

,count,pickup_zone,drop_off_zone,month,rank
0,1,Clinton East,TriBeCa/Civic Center,2008-12,1.0
1,6,Clinton West,Murray Hill,2008-12,1.0
2,2,Financial District South,Meatpacking/West Village West,2008-12,1.0
3,1,Gramercy,World Trade Center,2008-12,1.0
4,1,JFK Airport,Hamilton Heights,2008-12,1.0
...,...,...,...,...,...
34857,1,JFK Airport,Saint Albans,2019-09,1.0
34858,1,Greenwich Village North,Upper West Side South,2038-02,1.0
34859,1,West Village,Greenwich Village South,2038-02,1.0
34860,1,Yorkville East,Lenox Hill East,2038-02,1.0


In [199]:
# 2.i) Sample Queries
ranked[(ranked["pickup_zone"] == "Allerton/Pelham Gardens") & (ranked["rank"]==1.0)]

,count,pickup_zone,drop_off_zone,month,rank
170,26,Allerton/Pelham Gardens,Allerton/Pelham Gardens,2019-02,1.0


In [200]:
# 2.i) Sample Queries
# Which destination was reached by the most passengers from Chinatown in September 2019.
# grouped[(grouped["rank"]==1.0) & (grouped["pickup_zone"] == "Chinatown")]
ranked[(ranked["pickup_zone"] == "Chinatown") & (ranked["rank"]<=5.0) & (ranked["month"]=="2019-09")]

,count,pickup_zone,drop_off_zone,month,rank
